In [ ]:
!pip install -r ../requirements.txt

In [ ]:
from collections import Counter
import hashlib
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from joblib import dump

from ds_helpers import temporada_alta, dif_min, get_periodo_dia

## Manejo y transformación del dataset

In [ ]:
df = pd.read_csv('../datasets/dataset_SCL.csv')
df = df[:-1]
df

In [ ]:
df['temporada_alta'] = df['Fecha-I'].apply(temporada_alta)
df['dif_min'] = df.apply(dif_min, axis = 1)
df['atraso_15'] = np.where(df['dif_min'] > 15, 1, 0)
df['periodo_dia'] = df['Fecha-I'].apply(get_periodo_dia)

Dejar solo algunas columnas, de acuerdo al criterio del experto humano.

In [ ]:
data = df[['OPERA', 'MES', 'TIPOVUELO', 'SIGLAORI', 'SIGLADES', 'DIANOM','temporada_alta', 'atraso_15']]
label = data['atraso_15']

In [ ]:
data

Transformar los datos originales en una represetanción numérica simple que consiste en mapear cada valor de la variable categórica a un valor entero.

In [ ]:
features = data.assign(
    OPERA = LabelEncoder().fit_transform(data['OPERA']),
    MES = data.MES,
    TIPOVUELO = LabelEncoder().fit_transform(data['TIPOVUELO']),
    SIGLAORI = LabelEncoder().fit_transform(data['SIGLAORI']),
    SIGLADES = LabelEncoder().fit_transform(data['SIGLADES']),
    DIANOM = data['DIANOM'].map( {'Lunes':1, 'Martes':2, 'Miercoles':3, 'Jueves':4, 'Viernes':5, 'Sabado':6, 'Domingo':7}),
    TEMPALTA = data.temporada_alta
).drop(columns=['atraso_15', 'temporada_alta'])

features

Separar el dataset en train y test, con *0,67* para training  y *0,33* para testing. Se sigue una estrategia `stratified`, es decir, se mantiene la proporción de la variable objetivo (`label`) en train y test. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.33, stratify=label, random_state = 1)

Verificaciones varias para asegurarse que los 2 conjuntos tienen las propiedades requeridas

In [ ]:
assert y_train.size + y_test.size == label.size
assert x_train.shape[0] + x_test.shape[0] == features.shape[0]

In [ ]:
print(y_train.size)
print(y_test.size)
# this 2-value arrays must be close
print(y_train.value_counts('%').values)
print(y_test.value_counts('%').values)
np.allclose(y_train.value_counts('%').values, y_test.value_counts('%').values, atol=0.01)

## Entrenar modelo

Se entrena un modelo de predicción utilizando el algoritmo de aprendizaje *XGBoost*. Se elegió este método porque en la literatura ha demostrado obtener los mejores resultados en datos tabulares. Debido a que es un dataset desbalanceado (80% sin atraso y 20% con atraso), se utiliza el parámetro `scale_pos_weight` propio de algoritmo, para ponderar la distinta proporción de las clases.
Para identificar los parámetros que entregan el mejor desempeño, se utilizó el algoritmo de `GridSearch` o búsqueda exhaustiva sobre un conjunto de parámetros definidos por el experto humano. En cada iteración de parámetros se utilizó una validación cruzada (CV) de 3 *folds* con estrategia *stratified*

In [ ]:
counter = Counter(y_train)
cls_weight = counter[0] / counter[1]

modelxgb = xgb.XGBClassifier(scale_pos_weight=cls_weight, random_state=None)
parameters = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [10, 50, 100],
    'subsample': [0.1, 0.5, 0.9],
    'max_depth': [6, 20, 50, 100] 
}

modelxgb_GridCV = GridSearchCV(
    modelxgb,
    param_grid = parameters,
    cv = 3,
    n_jobs=-1,
    verbose=1).fit(x_train, y_train)

Para medir la capacidad que tiene el modelo para generalizar, se utiliza un conjunto de datos de validación que el modelo no ha "visto".

In [ ]:
y_pred_xgb_grid = modelxgb_GridCV.predict(x_test)
print(confusion_matrix(y_test, y_pred_xgb_grid))
print(classification_report(y_test, y_pred_xgb_grid))

In [ ]:
print(modelxgb_GridCV.best_params_)
print(modelxgb_GridCV.best_estimator_)

phash = hashlib.sha1(str(modelxgb_GridCV.best_estimator_.get_params()).encode('utf-8')).hexdigest() 
mname = f"{datetime.utcnow().strftime('%Y%m%d')}_{phash[:7]}_xgb_m1.joblib"

dump(modelxgb_GridCV.best_estimator_, f'../models/{mname}')

## Trabajo futuro

- Incorporar nuevas features que aporten al modelo
- Evaluar otra representación de los datos (e.g., one hot encoding) junto a otro algoritmo de aprendizaje (e.g., NN)